In [1]:
import pandas as pd
import numpy as np

_753 = 'submission_rmse_8.583118.csv'

sub1 = pd.read_csv("/kaggle/input/stu-scores-senet-lb8-60471/submission.csv")

sub2  = pd.read_csv("/kaggle/input/s6e1-8-54753-hill-climbing-ridgecv-stacking/" + _753)
sub33 = pd.read_csv("/kaggle/input/ps-s6e1-10/noscore8.csv")
sub44 = pd.read_csv("/kaggle/input/ps-s6e1-14/nscore15.csv")
sub55 = pd.read_csv("/kaggle/input/ps-s6e1-14/nscore16.csv")
sub7  = pd.read_csv("/kaggle/input/predicting-student-performance-in-exams/submission.csv")

df,t = pd.read_csv('/kaggle/input/playground-series-s6e1/sample_submission.csv'),'exam_score'

def blend(df1,df2,wts,file):
    df[t] = df1[t]*wts[0] + df2[t]*wts[1]
    df.to_csv(file,index=False)

blend( sub1, sub2, [0.025, 0.975], '2a.csv')
blend( sub1, sub33,[0.025, 0.975], '3a.csv')
blend( sub1, sub7, [0.026, 0.974], '7a.csv')

blend( sub1, sub2, [0.025, 0.975], '2b.csv')
blend( sub1, sub44,[0.025, 0.975], '3b.csv')
blend( sub1, sub7, [0.026, 0.974], '7b.csv')

blend( sub1, sub2, [0.025, 0.975], '2c.csv')
blend( sub1, sub55,[0.025, 0.975], '3c.csv')
blend( sub1, sub7, [0.026, 0.974], '7c.csv')

In [2]:
import numpy as np
import pandas as pd

import os,ast,shutil,copy,random

from bokeh.plotting import figure, gridplot 
from bokeh.io import output_file, show, output_notebook
output_notebook()


def bokeh_show(
        params,
        df_cross,
        show_figures1, 
        show_figures2, wps_fig2,
        color_cross):

    colors = [subm['color'] for subm in params['subm']]
    
    def dossier(js,subms,cols):
        def quant(i,js,subms,cols):
            return {"c" : i, "q" : sum([1 for subm in cols[i] if subm == subms[js]])}
        return {
            'name' : subms[js],
            'q_in' : [quant(i,js,subms,cols) for i in range(len(subms))]
        }
    alls = pd.read_csv(f'tida_desc.csv')
    matrix = [ast.literal_eval(str(row.alls)) for row in alls.itertuples()]
    subms = sorted(matrix[0])
    cols = [[data[i] for data in matrix] for i in range(len(subms))]
    df_subms = pd.DataFrame({f'col_{i}': [x[i] for x in matrix] for i in range(len(subms))})
    dossiers = [dossier(js,subms,cols) for js in range(len(subms))]
    subm_names = [one_dossier['name'] for one_dossier in dossiers]
    figures1,qss,i = [],[],0
    height = 100 if len(colors)==2\
        else 134 if len(colors)==3 else (154 if len(colors)==4 else 174)
    for one_dossier in dossiers: 
        i_col = 'alls. ' + str(one_dossier['q_in'][i]['c'])
        qs = [one['q'] for one in one_dossier['q_in']]
        x_names = [name.replace("Group","").replace("subm_","") for name in subm_names]
        width = 140
        f = figure(x_range=x_names,width=width, height=height, title=i_col)
        f.vbar(x=x_names, width=0.585, top=qs, color=colors)
        figures1.append(f)
        qss.append(qs)
        i+=1
    grid = gridplot([figures1])
    output_file('tida_alls.html')
    if show_figures1 == True: show(grid)
    sub_wts = params['subwts']
    main_wts = [subm['weight'] for subm in params['subm']]
    mms,acc_mass = [],[]
    for j in range(len(dossiers)):
        one_dossier = dossiers[j]
        qs = [one['q'] for one in one_dossier['q_in']]
        mm = [qs[h] * (main_wts[j] + sub_wts[h]) for h in range(len(sub_wts))]
        mass = sum(mm)
        mms.append(mm)
        acc_mass.append(round(mass))                        #subm_names[::-1]
    y_names = [name + " - " + str(mass) for name,mass in zip(subm_names,acc_mass)]
    f1 = figure(y_range=y_names, width=270, height=height, title='relations of general masses')
    f1.hbar(y=y_names, height=0.555, right=acc_mass, left=0, color=colors)
    output_file('tida_alls2.html')
    alls = [f'alls.{i}' for i in range(len(dossiers))]
    subm = [f'sub{i}'   for i in range(len(dossiers))] 
    mmsT  = np.asarray(mms).T
    data = {'cols' : alls}
    for i in range(len(dossiers)): data[f'sub{i}'] = mmsT[i,:]
    f2 = figure(y_range=alls, height=height, width=270, title="relations of columns masses")
    f2.hbar_stack(subm, y='cols', height=0.555, color=colors, source=data)
    qssT  = np.asarray(qss).T
    data = {'cols' : alls}
    for i in range(len(dossiers)): data[f'sub{i}'] = qssT[i,:]
    f3 = figure(y_range=alls, height=height, width=245, title="ratios in columns")
    f3.hbar_stack(subm, y='cols', height=0.555, color=colors, source=data)
    grid = gridplot([[f3,f2,f1]])
    show(grid)
    if show_figures2 == True:
        def read(params,i):
            FiN = params["path"] + params["subm"][i]["name"] + ".csv"
            target_name_back = {'target':params["target"],'pred':params["target"]}
            return pd.read_csv(FiN).rename(columns=target_name_back)
        dfs = [read(params,i) for i in range(len(params["subm"]))] + [df_cross]
        _height = 358 if len(params["subm"]) == 11 else 254
        f   = figure(width=785, height=_height)
        f.title.text = 'Click on legend entries to mute the corresponding lines'
        b,e        = 21000,21154
        line_x     = [dfs[i][b:e]['id']         for i in range(len(dfs))]
        line_y     = [dfs[i][b:e]['exam_score'] for i in range(len(dfs))]
        color      = colors + [color_cross]
        alpha      = [0.8 for i in range(len(dfs)-1)] + [0.95]
        lws        = [1.0 for i in range(len(dfs)-1)] + [1.00]
        legend = subm_names + ['cross']
        for i in range(len(legend)):
            f.line(line_x[i], line_y[i], line_width=lws[i], color=color[i], alpha=alpha[i],
                   muted_color='white',legend_label=legend[i])
        f.legend.location = "top_left"
        f.legend.click_policy="mute"
        show(f)


def matrix_vs(path,fs_names):
    def load(path,fs_names):
        dfs = [pd.read_csv(path + name_subm +'.csv') for name_subm in fs_names]
        for i in range(len(dfs)):
            dfs[i] = dfs[i].rename(columns={"exam_score": f'{fs_names[i]}'})
        dfsm = pd.merge(dfs[0], dfs[1], on="id")
        for i in range(2,len(dfs)):
            dfsm = pd.merge(dfsm,dfs[i],on='id')
        return dfsm   
    def make_list_vs(fs_names):
        list = []
        for i in range(0,len(fs_names)-1):
            for j in range(i+1,len(fs_names)):
                list.append(fs_names[i] + "_vs_" + fs_names[j])
        return list
    def get_mvs(dfs, list_vs):
        def get_abs_distance(x,t1,t2):
            return abs(x[t1]-x[t2])
        for vs in list_vs:
            t = vs.split('_vs_')
            dfs[vs] = dfs.apply(lambda x: get_abs_distance(x,t[0],t[1]), axis=1)
        return dfs   
    def distance_vs(name, st_names, list_vs, dfs):
        distances = []
        for st in st_names:
            vs_between = name + "_vs_" + st
            if vs_between not in list_vs:
                distances.append(0)
            else: distances.append(round(dfs[vs_between].sum()))
        return distances
    dfs = load(path,fs_names)
    list_vs = make_list_vs(fs_names)
    mvs = get_mvs(dfs, list_vs)
    m1 = pd.DataFrame({'subm':fs_names})
    m2 = pd.DataFrame({ name :distance_vs(name, fs_names, list_vs, mvs) for name in fs_names})
    matrix = pd.concat([m1,m2],axis=1)
    return matrix


def seaborn_Show(params,file_name_cross=''):
    import matplotlib.pyplot as plt, seaborn as sns
    import warnings; warnings.filterwarnings('ignore')
    plt.figure(figsize=(8.7, 2))
    for subm in params['subm']:
        pred = pd.read_csv(params['path']+subm['name']+'.csv')[params['id_target'][1]]
        sns.kdeplot(pred, label = subm['name'], linewidth = 0.5)
    if file_name_cross != '':
        pred = pd.read_csv(file_name_cross)[params['id_target'][1]]
        sns.kdeplot(pred, label = 'blend', linewidth = 1, linestyle = 'dashed')
    plt.title("KDE")
    plt.xlabel("target")
    plt.ylabel("Density")
    plt.legend()
    plt.grid(alpha=0.3)
    plt.show()


# df = df_A.merge(df_B,on='id').merge(df_C,on='id')
# df = df.rename(columns={'exam_score_x':'A','exam_score_y':'B','exam_score':'C'})
# df


def display_distances(params):
    files = [subm['name'] for subm in params['subm']]
    distances = matrix_vs ( params['path'], files )            
    display(distances)


def arr_colors(color):
    dskb,mvr = 'deepskyblue','mediumvioletred'
    sg = ['darkgray','silver','gainsboro']
    if color=='red'   or color=='R': return ['firebrick','red','crimson','tomato']     + sg
    if color=='Red'   or color=='r': return ['red','tomato','crimson']                 + sg
    if color=='Green' or color=='G': return ['darkgreen','limegreen','green','lime']   + sg
    if color=='Blue'  or color=='B': return ['midnightblue','blue','mediumblue',dskb]  + sg
    if color=='RGB'   or color=='S': return ['mediumblue','darkgreen','crimson']       + sg
    if color=='RGBM'  or color=='M': return [mvr,'darkorchid','darkmagenta','magenta'] + sg
    return ['black','dimgray','gray'] + sg


def convert(schema):
    colors = arr_colors(schema[2])
    dicts  = [
        {'name': schema[0][i],'weight':schema[1][i],'color':colors[i]} 
        for i in range(len(schema[0]))
    ]
    return {'subm':dicts}


def h_blend(
        params, _update={},
        cross='silver',
        details=False,
        fig1=False, fig2=False, wf2=555, 
        dtls=False, dist=False, subm='',tida=''):

    if isinstance(params, list): params = convert(params)

    if 'path' in _update: params.update(_update)
    
    color_cross, dk  = cross, copy.deepcopy(params)

    if details == True:
        dist = True
        show_details,show_figures1,show_figures2 = True,True,True
    else:
        show_details,show_figures1,show_figures2 = dtls,fig1,fig2
        
    file_short_names = [subm['name'] for subm in params['subm']]
    type_sort    = params['type_sort'][0]
    dk['asc']    = params['type_sort'][1]
    dk['desc']   = params['type_sort'][2]
    dk['id']     = params['id_target'][0]
    dk['target'] = params['id_target'][1]
# ------------------------------------------------------------------------
    def read(dk,i):
        tnm = dk["subm"][i]["name"]
        FiN = dk["path"] + tnm + ".csv"
        return pd.read_csv(FiN).rename(columns={
            'target':tnm, 'pred':tnm, dk["target"]:tnm})
        
    def merge(dfs_subm):
        df_subms = pd.merge(dfs_subm[0],  dfs_subm[1], on=[dk['id']])
        for i in range(2, len(dk["subm"])): 
            df_subms = pd.merge(df_subms, dfs_subm[i], on=[dk['id']])
        return df_subms
        
    def da(dk,sorting_direction,show_details):
        
        df_subms = merge([read(dk,i) for i in range(len(dk["subm"]))])
        cols = [col for col in df_subms.columns if col != dk['id']]
        short_name_cols = [c for c in cols]
        
        def alls1(x, sd=sorting_direction,cs=cols):
            reverse = True if sd=='desc' else False
            tes = {c: x[c] for c in cs}.items()
            subms_sorted = [t[0] for t in sorted(tes,key=lambda k:k[1],reverse=reverse)]
            return subms_sorted

        def alls2(x, sd=sorting_direction,cs=cols):
            reverse = True if sd=='desc' else False
            tes = {c: x[c] for c in cs}.items()
            subms_random = [t[0] for t in tes]
            random.shuffle(subms_random)
            return subms_random

        alls = alls1 if type_sort == 'asc/desc' else alls2
            
        def summa(x,cs,wts,ic_alls): 
            return sum([x[cs[j]] * (wts[0][j] + wts[1][ic_alls[j]]) for j in range(len(cs))])
            
        wts = [[[e['weight'] for e in dk["subm"]], [w for w in dk["subwts"]]]]
          
        def correct(x, cs=cols, wts=wts):
            i = [x['alls'].index(c) for c in short_name_cols]
            return summa(x,cs,wts[0],i)

        if len(wts) == 1:
            correct_sub_weights = [wt for wt in dk["subwts"]]
            weights = [subm['weight'] for subm in dk["subm"]]
            def correct(x, cs=cols, w=weights, cw=correct_sub_weights):
                ic = [x['alls'].index(c) for c in short_name_cols]
                cS = [x[cols[j]] * (w[j] + cw[ic[j]]) for j in range(len(cols))]
                return sum(cS)
                
        if len(wts) > 1 or "subwts2" in dk:

            wts = [
                [[e['weight'] for e in dk["subm"]], [w for w in dk["subwts" ]]],
                [[e['weight'] for e in dk["subm2"]],[w for w in dk["subwts2"]]],
                [[e['weight'] for e in dk["subm3"]],[w for w in dk["subwts3"]]],
            ]

            def correct(x, cs=cols, wts=wts):
                i = [x['alls'].index(c) for c in short_name_cols]
                if   0.0540 < x['mx-m'] <= 0.0740: return summa(x,cs,wts[2],i)
                if   0.0000 < x['mx-m'] <= 0.0050: return summa(x,cs,wts[1],i)
                else:                              return summa(x,cs,wts[0],i)
                   
        def amxm(x, cs=cols):
            list_values = x[cs].to_list()
            mxm = abs(max(list_values)-min(list_values))
            return mxm

        if len(wts) > 1 or "subwts2" in dk: # or True:
            df_subms['mx-m']   = df_subms.apply(lambda x: amxm   (x), axis=1)
        df_subms['alls']       = df_subms.apply(lambda x: alls   (x), axis=1)
        df_subms[dk["target"]] = df_subms.apply(lambda x: correct(x), axis=1)
        schema_rename = { old_nc:new_shnc for old_nc, new_shnc in zip(cols, short_name_cols) }
        df_subms = df_subms.rename(columns=schema_rename)
        df_subms = df_subms.rename(columns={dk["target"]:"ensemble"})
        df_subms.insert(loc=1, column=' _ ', value=['   '] * len(df_subms))
        df_subms[' _ '] = df_subms[' _ '].astype(str)
        pd.set_option('display.max_rows',100)
        pd.set_option('display.float_format', '{:.5f}'.format)
        if len(wts) > 1: # or True: 
            vcols = [dk['id']] + [' _ '] + short_name_cols + [' _ '] + ['mx-m'] + [' _ '] +\
                      ['alls'] + [' _ '] + ['ensemble']
        else:
            vcols = [dk['id']] + [' _ '] + short_name_cols + [' _ '] +\
                      ['alls'] + [' _ '] + ['ensemble']
        df_subms = df_subms[vcols]
        if show_details and sorting_direction=='desc': display(df_subms.head(5))
        pd.set_option('display.float_format', '{:.5f}'.format)
        df_subms = df_subms.rename(columns={"ensemble":dk["target"]})
        if sorting_direction=='desc': 
            df_subms.to_csv(f'tida_{sorting_direction}.csv', index=False)
        return df_subms[[dk['id'],dk['target']]]
   
    def ensemble_da(dk,        show_details): 
        dfD    = da(dk,'desc', show_details)
        dfA    = da(dk,'asc',  show_details)
        dfA[dk['target']] = dk['desc']*dfD[dk['target']] + dfA[dk['target']]*dk['asc']
        return dfA

    da = ensemble_da(dk,show_details)
                                    # show_figures2,
    bokeh_show(dk, da, show_figures1, False,         wf2, color_cross)

    if subm != '': da.to_csv(subm, index=False)

    print('distances between participiants'); display_distances(params)
    
    # if dist == True: 
    #     added = False
    #     if subm != '' and '/kaggle/working/' in params['path']: 
    #         params['subm'].append({'name':subm.replace('.csv','').replace(params['path'],'')})
    #         added = True
    #     print('distances between participiants'); display_distances(params)
    #     cross_FiN = subm if not added else ''
    #     if len(params['subm']) not in [4,5]:
    #         seaborn_Show(params, file_name_cross=cross_FiN)

    if tida != '':
        dt = pd.read_csv('/kaggle/working/tida_desc.csv')
        dt['ensemble'] = da['exam_score']
        dt[['mx-m','ensemble']].to_csv(tida,index=False)
        #display(dt[['mx-m','ensemble']].head(7))

    return  da


# def blend2(fin0,fin1,wts,out):
#     path_to_ds = '/kaggle/input/ps-s6e1-08/'
#     df = pd.read_csv('/kaggle/input/playground-series-s6e1/sample_submission.csv')
#     df0 = pd.read_csv(path_to_ds + fin0 + '.csv')
#     df1 = pd.read_csv(path_to_ds + fin1 + '.csv')
#     df['exam_score'] = \
#         df0['exam_score'] * wts[0] +\
#         df1['exam_score'] * wts[1]
#     df.to_csv(out,index=False)                  
#     return df   

Loading BokehJS ...

In [3]:
wts = [ 0.10, 0.45, 0.45 ]

In [4]:
params = {
       'path'     : '/kaggle/working/',
       'id_target': ['id',"exam_score"],
       'type_sort': ['asc/desc',0.30,0.70 ],
       'subwts'   : [w/100 for w in [+10,-7,-3]],
       'subm'     : [
           {'name': f'2a','weight':wts[0],'color':'midnightblue'},
           {'name': f'3a','weight':wts[1],'color':'blue'},
           {'name': f'7a','weight':wts[2],'color':'mediumblue'},]
}   
df_a = h_blend(params, details=True, subm='a.csv')

,id,_,2a,3a,7a,_,alls,_,ensemble
0,630000,,69.40763,69.65285,69.29440,,"[3a, 2a, 7a]",,69.49494
1,630001,,67.51100,67.26390,67.67995,,"[7a, 2a, 3a]",,67.50014
2,630002,,90.59108,90.49635,90.69433,,"[7a, 2a, 3a]",,90.60808
3,630003,,56.48426,56.58015,56.76916,,"[7a, 3a, 2a]",,56.67740
4,630004,,46.48701,46.63519,46.70818,,"[7a, 3a, 2a]",,46.66496


distances between participiants


,subm,2a,3a,7a
0,2a,0,0,0
1,3a,41640,0,0
2,7a,39427,55400,0


In [5]:
params = {
       'path'     : '/kaggle/working/',
       'id_target': ['id',"exam_score"],
       'type_sort': ['asc/desc',0.30,0.70 ],
       'subwts'   : [w/100 for w in [+10,-7,-3]],
       'subm'     : [
           {'name': f'2b','weight':wts[0],'color':'midnightblue'},
           {'name': f'3b','weight':wts[1],'color':'blue'},
           {'name': f'7b','weight':wts[2],'color':'mediumblue'},]
} 
df_b = h_blend(params, details=True, subm='b.csv')

,id,_,2b,3b,7b,_,alls,_,ensemble
0,630000,,69.40763,69.32814,69.29440,,"[2b, 3b, 7b]",,69.32987
1,630001,,67.51100,67.61270,67.67995,,"[7b, 3b, 2b]",,67.64257
2,630002,,90.59108,90.60422,90.69433,,"[7b, 3b, 2b]",,90.65286
3,630003,,56.48426,56.66421,56.76916,,"[7b, 3b, 2b]",,56.70934
4,630004,,46.48701,46.68188,46.70818,,"[7b, 3b, 2b]",,46.68271


distances between participiants


,subm,2b,3b,7b
0,2b,0,0,0
1,3b,42444,0,0
2,7b,39427,14708,0


In [6]:
params = {
       'path'     : '/kaggle/working/',
       'id_target': ['id',"exam_score"],
       'type_sort': ['asc/desc',0.30,0.70 ],
       'subwts'   : [w/100 for w in [+10,-7,-3]],
       'subm'     : [
           {'name': f'2c','weight':wts[0],'color':'midnightblue'},
           {'name': f'3c','weight':wts[1],'color':'blue'},
           {'name': f'7c','weight':wts[2],'color':'mediumblue'},]
}   
df_c = h_blend(params, details=True, subm='c.csv')

,id,_,2c,3c,7c,_,alls,_,ensemble
0,630000,,69.40763,69.33746,69.29440,,"[2c, 3c, 7c]",,69.33341
1,630001,,67.51100,67.61634,67.67995,,"[7c, 3c, 2c]",,67.64395
2,630002,,90.59108,90.58950,90.69433,,"[7c, 2c, 3c]",,90.64720
3,630003,,56.48426,56.66518,56.76916,,"[7c, 3c, 2c]",,56.70971
4,630004,,46.48701,46.70361,46.70818,,"[7c, 3c, 2c]",,46.69096


distances between participiants


,subm,2c,3c,7c
0,2c,0,0,0
1,3c,42395,0,0
2,7c,39427,14830,0


In [7]:
params = {
       'path'     : '/kaggle/working/',
       'id_target': ['id',"exam_score"],
       'type_sort': ['asc/desc',0.30,0.70 ],
       'subwts'   : [w/100 for w in [+10,-7,-3]],
       'subm'     : [
           {'name': f'a','weight':0.20,'color':'navy'},
           {'name': f'b','weight':0.40,'color':'crimson'},
           {'name': f'c','weight':0.40,'color':'mediumblue'}, ]
}  
df_c = h_blend(params, details=True)

,id,_,a,b,c,_,alls,_,ensemble
0,630000,,69.48096,69.32545,69.32899,,"[a, c, b]",,69.37327
1,630001,,67.48392,67.63598,67.63736,,"[c, b, a]",,67.61082
2,630002,,90.60036,90.64883,90.64311,,"[b, c, a]",,90.63870
3,630003,,56.66629,56.69823,56.69860,,"[c, b, a]",,56.69298
4,630004,,46.65633,46.67408,46.68233,,"[c, b, a]",,46.67519


distances between participiants


,subm,a,b,c
0,a,0,0,0
1,b,24197,0,0
2,c,24227,871,0


In [8]:
for i in 'a,2a,3a,7a, b,2b,3b,7b, c.2c,3c,7b,'.split(','):
    file = f'/kaggle/working/{i}.csv'
    if os.path.isfile(file): os.remove(file)
        
os.remove('/kaggle/working/tida_desc.csv')

In [9]:
df.to_csv('submission.csv',index=False)
df

,id,exam_score
0,630000,69.29440
1,630001,67.67995
2,630002,90.69433
3,630003,56.76916
4,630004,46.70818
...,...,...
269995,899995,62.25737
269996,899996,44.76182
269997,899997,93.71675
269998,899998,53.02136
